In [ ]:
pip install backtrader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 33.3 MB/s 


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import backtrader as bt
import yfinance as yf

In [ ]:
data=bt.feeds.PandasData(dataname=yf.download('MSFT','2020-01-01','2021-01-01'))

[*********************100%***********************]  1 of 1 completed


In [ ]:
data

strategy:
1.Check for  MACD crossover
2. Wait till rsi exceeds 40
3.Wait till traded volume exceeds sms volume
4. Sell for downward macd crossover and rsi<80 and traded volume is less than sma volume

In [ ]:
class Strategy(bt.Strategy):
    def __init__(self):
        super().__init__()
        self.rsi=bt.indicators.RSI_Safe(self.data.close,period=14)
        self.volume=self.data.volume
        self.volume_sma=bt.indicators.Average(self.volume,period=14)
        self.me1=bt.indicators.EMA(self.data.close,period=26)
        self.me2=bt.indicators.EMA(self.data.close,period=12)
        self.macd=self.me1-self.me2
        self.signal=bt.indicators.EMA(self.macd,period=9)
        self.diff=self.macd-self.signal
        self.long=False
        self.stock=False
        self.diffsignal=False
        self.rsisignal=False
    
        
        
        
        
        
    def log(self,text):
        dt=self.datas[0].datetime.date(0)
        print(dt.isoformat(),', '+text)
        
        
    def notify_order(self,order):
        if order.status in [order.Submitted,order.Accepted]:
            return
        if order.status in[order.Completed]:
            if order.isbuy():
                self.log('Buy Executed: Price-%2f,Cost-%2f,Commission-%2f' % (order.executed.price,order.executed.value,order.executed.comm))
                self.long=True
                self.diffsignal=False
                self.stock=False
                self.rsisignal=False
                
            if order.issell():
                self.log('Sell Executed: Price-%2f,Cost-%2f,Commission-%2f' % (order.executed.price,order.executed.value,order.executed.comm))
                self.long=False
                self.diffsignal=False
                self.stock=False
                self.rsisignal=False
                
            self.bar_executed=len(self)
            
        elif order.status in [order.Canceled,order.Margin,order.Rejected]:
            self.log('Order rejected/margin/cancelled')
            
            
        self.order=None
    
    
    def next(self):
        if not self.long:
            if not self.diffsignal:
                if self.diff[-1]<0 and self.diff[0]>0:
                    self.diffsignal=True
            if self.diffsignal:
                if self.diff[0]>0:
                    self.stock=True
                else:
                    self.diffsignal=False
                    self.stock=False
                    self.rsisignal=False
            
            if self.stock:
                if self.rsi>40:
                    self.rsisignal=True
                else:
                    self.rsisignal=False
        
            if self.rsisignal:
                if self.volume>self.volume_sma:
                    self.buy()

                
                
                
                    
        if self.long:
            if not self.diffsignal:
                if self.diff[-1]>0 and self.diff[0]<0:
                    self.diffsignal=True
            if self.diffsignal:
                if self.diff[0]<0:
                    self.stock=True
                else:
                    self.diffsignal=False
                    self.stock=False
                    self.rsisignal=False
                    
            if self.stock:
                if self.rsi<80:
                    self.rsisignal=True
                else:
                    self.rsisignal=False
                    
            if self.rsisignal:
                if self.volume>self.volume_sma:
                    self.sell()
                
               
                    
            
            
            
        
    def stop(self):
        if self.long:
            self.sell()
    

In [ ]:
cerebro=bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(Strategy)
cerebro.broker.setcash(1000000.0)
cerebro.addsizer(bt.sizers.FixedSize,stake=4000)
cerebro.broker.setcommission(commission=0.005)
cerebro.addanalyzer(bt.analyzers.SharpeRatio,_name='mysharpe',compression=60,timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(bt.analyzers.DrawDown,_name='mydrawdown')
cerebro.addanalyzer(bt.analyzers.DrawDown)
strategies=cerebro.run()
print('Final cash is- %2f' % (cerebro.broker.get_value()))



2020-05-14 , Buy Executed: Price-177.539993,Cost-710159.973145,Commission-3550.799866
2020-06-11 , Sell Executed: Price-193.130005,Cost-710159.973145,Commission-3862.600098
2020-07-21 , Buy Executed: Price-213.660004,Cost-854640.014648,Commission-4273.200073
2020-08-05 , Sell Executed: Price-214.899994,Cost-854640.014648,Commission-4297.999878
2020-09-08 , Buy Executed: Price-206.500000,Cost-826000.000000,Commission-4130.000000
2020-10-01 , Sell Executed: Price-213.490005,Cost-826000.000000,Commission-4269.800110
2020-10-27 , Buy Executed: Price-211.589996,Cost-846359.985352,Commission-4231.799927
2020-11-06 , Sell Executed: Price-222.259995,Cost-846359.985352,Commission-4445.199890
2020-11-25 , Buy Executed: Price-215.110001,Cost-860440.002441,Commission-4302.200012
2020-12-02 , Sell Executed: Price-214.880005,Cost-860440.002441,Commission-4297.600098
2020-12-10 , Buy Executed: Price-211.770004,Cost-847080.017090,Commission-4235.400085
2020-12-17 , Sell Executed: Price-219.869995,Cost

In [ ]:
strategies[0].analyzers.mysharpe.get_analysis()


OrderedDict([('sharperatio', 0.05553454274643205)])

In [ ]:
strategies[0].analyzers.mydrawdown.get_analysis()

AutoOrderedDict([('len', 10),
                 ('drawdown', 0.18172000404145594),
                 ('moneydown', 2037.41455078125),
                 ('max',
                  AutoOrderedDict([('len', 102),
                                   ('drawdown', 6.744825740122326),
                                   ('moneydown', 72415.76843261719)]))])